##Lab 6 
---
###E20CSE314
###EB15

In [181]:
import pandas as pd
import numpy as np
import sklearn
import collections
from collections import Counter
from sklearn.datasets import make_classification

from imblearn.over_sampling import SMOTE,SMOTENC,SMOTEN,BorderlineSMOTE,RandomOverSampler,ADASYN,KMeansSMOTE,SVMSMOTE
from imblearn.under_sampling import NearMiss,TomekLinks,RandomUnderSampler,NeighbourhoodCleaningRule,OneSidedSelection

from collections import Counter
from sklearn.datasets import make_classification
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils
import pylab as plt
import tensorflow as tf

In [126]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data

--2023-03-29 15:20:46--  http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18654 (18K) [application/x-httpd-php]
Saving to: ‘breast-cancer.data.3’

breast-cancer.data. 100%[===================>]  18.22K   118KB/s    in 0.2s    

2023-03-29 15:20:47 (118 KB/s) - ‘breast-cancer.data.3’ saved [18654/18654]



In [127]:
DF=pd.read_csv('breast-cancer.data',names=['Class','age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'],na_values='?')

In [128]:
DF.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [129]:
DF.shape


(286, 10)

In [130]:
DF.isnull().sum()


Class          0
age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      8
deg-malig      0
breast         0
breast-quad    1
irradiat       0
dtype: int64

In [131]:
sum(DF.Class=='no-recurrence-events')

201

In [132]:
sum(DF.Class=='recurrence-events')

85

In [133]:
DF.isnull().sum()

Class          0
age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      8
deg-malig      0
breast         0
breast-quad    1
irradiat       0
dtype: int64

In [134]:
for column in DF.columns:
    DF[column].fillna(DF[column].mode()[0], inplace=True)

In [135]:
DF.isnull().sum()

Class          0
age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      0
deg-malig      0
breast         0
breast-quad    0
irradiat       0
dtype: int64

In [136]:
pd.get_dummies(DF)

,deg-malig,Class_no-recurrence-events,Class_recurrence-events,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,...,node-caps_yes,breast_left,breast_right,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,3,1,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1,2,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
2,2,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
3,2,1,0,0,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
4,2,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
282,3,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
283,1,0,1,0,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
284,3,0,1,0,0,1,0,0,0,1,...,0,1,0,0,1,0,0,0,1,0


In [137]:
y=DF['Class']

In [138]:
X=DF.drop(['Class'],axis=1)

In [139]:
X

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...
281,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [140]:
y

0      no-recurrence-events
1      no-recurrence-events
2      no-recurrence-events
3      no-recurrence-events
4      no-recurrence-events
               ...         
281       recurrence-events
282       recurrence-events
283       recurrence-events
284       recurrence-events
285       recurrence-events
Name: Class, Length: 286, dtype: object

In [141]:
y=y=='no-recurrence-events'

In [142]:
y=y.astype(int)

In [143]:
X=pd.get_dummies(X,sparse=False)

In [144]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=2)

In [145]:
def build(X_train,y_train):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_shape=(39,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  hist = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=2)
  return hist,model

####No Sampling methods used

In [146]:
histnone,modelnone = build(X_train,y_train)

score = modelnone.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 32)                1280      
                                                                 
 dense_41 (Dense)            (None, 64)                2112      
                                                                 
 dense_42 (Dense)            (None, 32)                2080      
                                                                 
 dense_43 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
8/8 - 2s - loss: 0.7017 - accuracy: 0.4868 - 2s/epoch - 280ms/step
Epoch 2/10
8/8 - 0s - loss: 0.6379 - accuracy: 0.6974 - 42ms/epoch - 5ms/step
Epoch 3/10
8/8 - 0s - loss:

In [147]:
X=pd.get_dummies(X,sparse=True)

## Oversampling

###RandomOverSampling

In [148]:
ros = RandomOverSampler(random_state=42)
X_ReOsam, y_ReOsam = ros.fit_resample(X, y)

In [149]:
X_train,X_test,y_train,y_test= train_test_split(X_ReOsam,y_ReOsam,test_size=0.2,random_state=2)

In [150]:
hist1,model1=build(X_train,y_train)
score = model1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 32)                1280      
                                                                 
 dense_45 (Dense)            (None, 64)                2112      
                                                                 
 dense_46 (Dense)            (None, 32)                2080      
                                                                 
 dense_47 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 4s - loss: 0.6792 - accuracy: 0.5327 - 4s/epoch - 358ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6586 - accuracy: 0.5857 - 57ms/epoch - 5ms/step
Epoch 3/10
11/11 - 0s -

###SMOTE

In [151]:
smote = SMOTE()
X_smote,y_smote = smote.fit_resample(X,y)

In [152]:
X_train,X_test,y_train,y_test= train_test_split(X_smote,y_smote,test_size=0.2,random_state=2)

In [153]:
hist2,model2=build(X_train,y_train)
score = model2.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 32)                1280      
                                                                 
 dense_49 (Dense)            (None, 64)                2112      
                                                                 
 dense_50 (Dense)            (None, 32)                2080      
                                                                 
 dense_51 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 3s - loss: 0.6809 - accuracy: 0.6012 - 3s/epoch - 263ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6604 - accuracy: 0.6542 - 60ms/epoch - 5ms/step
Epoch 3/10
11/11 - 0s -

###SMOTENC

In [154]:
smotenc = SMOTENC(categorical_features=[1,2,3,4,5,6,7,8], random_state=42)
X_smotenc,y_smotenc = smotenc.fit_resample(X,y)

In [155]:
X_train,X_test,y_train,y_test= train_test_split(X_smotenc,y_smotenc,test_size=0.2,random_state=2)

In [156]:
hist3,model3=build(X_train,y_train)
score = model3.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 32)                1280      
                                                                 
 dense_53 (Dense)            (None, 64)                2112      
                                                                 
 dense_54 (Dense)            (None, 32)                2080      
                                                                 
 dense_55 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 3s - loss: 0.6792 - accuracy: 0.6106 - 3s/epoch - 272ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6511 - accuracy: 0.6511 - 60ms/epoch - 5ms/step
Epoch 3/10
11/11 - 0s -

###SMOTEN

In [157]:
smoten = SMOTEN()
X_smoten,y_smoten = smoten.fit_resample(X,y)

In [158]:
X_train,X_test,y_train,y_test= train_test_split(X_smoten,y_smoten,test_size=0.2,random_state=2)

In [159]:
hist4,model4=build(X_train,y_train)
score = model4.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 32)                1280      
                                                                 
 dense_57 (Dense)            (None, 64)                2112      
                                                                 
 dense_58 (Dense)            (None, 32)                2080      
                                                                 
 dense_59 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 2s - loss: 0.7049 - accuracy: 0.5078 - 2s/epoch - 153ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6627 - accuracy: 0.6542 - 47ms/epoch - 4ms/step
Epoch 3/10
11/11 - 0s -

###ADASYN

In [160]:
adasyn= ADASYN()
X_ada, y_ada = adasyn.fit_resample(X,y)

In [161]:
X_train,X_test,y_train,y_test= train_test_split(X_ada,y_ada,test_size=0.2,random_state=2)

In [162]:
hist5,model5=build(X_train,y_train)
score = model5.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 32)                1280      
                                                                 
 dense_61 (Dense)            (None, 64)                2112      
                                                                 
 dense_62 (Dense)            (None, 32)                2080      
                                                                 
 dense_63 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 - 1s - loss: 0.6944 - accuracy: 0.5220 - 1s/epoch - 133ms/step
Epoch 2/10
10/10 - 0s - loss: 0.6810 - accuracy: 0.5975 - 36ms/epoch - 4ms/step
Epoch 3/10
10/10 - 0s -

###BorderlineSMOTE

In [163]:
borderlineSMOTE= BorderlineSMOTE()
X_bsmo, y_bsmo = borderlineSMOTE.fit_resample(X,y)

In [164]:
X_train,X_test,y_train,y_test= train_test_split(X_bsmo,y_bsmo,test_size=0.2,random_state=2)

In [165]:
hist6,model6=build(X_train,y_train)
score = model6.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 32)                1280      
                                                                 
 dense_65 (Dense)            (None, 64)                2112      
                                                                 
 dense_66 (Dense)            (None, 32)                2080      
                                                                 
 dense_67 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 2s - loss: 0.6793 - accuracy: 0.5483 - 2s/epoch - 225ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6534 - accuracy: 0.6324 - 40ms/epoch - 4ms/step
Epoch 3/10
11/11 - 0s -

###KMeansSMOTE

In [171]:
kmsmote = KMeansSMOTE(random_state=48)
X_kms,y_kms = kmsmote.fit_resample(X,y)

In [172]:
X_train,X_test,y_train,y_test= train_test_split(X_kms,y_kms,test_size=0.2,random_state=2)

In [173]:
hist7,model7=build(X_train,y_train)
score = model7.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_76 (Dense)            (None, 32)                1280      
                                                                 
 dense_77 (Dense)            (None, 64)                2112      
                                                                 
 dense_78 (Dense)            (None, 32)                2080      
                                                                 
 dense_79 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 1s - loss: 0.6960 - accuracy: 0.5202 - 1s/epoch - 121ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6337 - accuracy: 0.7632 - 37ms/epoch - 3ms/step
Epoch 3/10
11/11 - 0s -

###SVMSMOTE

In [174]:
svmsmote = SVMSMOTE()
X_svms,y_svms = svmsmote.fit_resample(X,y)

In [175]:
X_train,X_test,y_train,y_test= train_test_split(X_svms,y_svms,test_size=0.2,random_state=2)

In [176]:
hist8,model8=build(X_train,y_train)
score = model8.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 32)                1280      
                                                                 
 dense_81 (Dense)            (None, 64)                2112      
                                                                 
 dense_82 (Dense)            (None, 32)                2080      
                                                                 
 dense_83 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 - 1s - loss: 0.6818 - accuracy: 0.5234 - 1s/epoch - 119ms/step
Epoch 2/10
11/11 - 0s - loss: 0.6544 - accuracy: 0.6791 - 42ms/epoch - 4ms/step
Epoch 3/10
11/11 - 0s -

#Undersampling

###Near-Miss

In [177]:
nm = NearMiss()
X_nm, y_nm = nm.fit_resample(X, y)

In [178]:
X_train, X_test, y_train, y_test = train_test_split(X_nm, y_nm, test_size=0.2, random_state=42)

In [179]:
hist9,model9=build(X_train,y_train)
score = model9.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 32)                1280      
                                                                 
 dense_85 (Dense)            (None, 64)                2112      
                                                                 
 dense_86 (Dense)            (None, 32)                2080      
                                                                 
 dense_87 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
5/5 - 1s - loss: 0.7207 - accuracy: 0.4632 - 1s/epoch - 277ms/step
Epoch 2/10
5/5 - 0s - loss: 0.6763 - accuracy: 0.5809 - 22ms/epoch - 4ms/step
Epoch 3/10
5/5 - 0s - loss:

###NeighbourhoodCleaningRule

In [190]:
ncr=NeighbourhoodCleaningRule()
X_ncr,y_ncr = ncr.fit_resample(X,y)

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X_ncr, y_ncr, test_size=0.2, random_state=42)

In [193]:
hist10,model10=build(X_train,y_train)
score = model10.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_100 (Dense)           (None, 32)                1280      
                                                                 
 dense_101 (Dense)           (None, 64)                2112      
                                                                 
 dense_102 (Dense)           (None, 32)                2080      
                                                                 
 dense_103 (Dense)           (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
6/6 - 2s - loss: 0.7174 - accuracy: 0.5170 - 2s/epoch - 414ms/step
Epoch 2/10
6/6 - 0s - loss: 0.6658 - accuracy: 0.6364 - 24ms/epoch - 4ms/step
Epoch 3/10
6/6 - 0s - loss:

###OneSidedSelection

In [194]:
oss = OneSidedSelection(random_state=17)
X_oss,y_oss=oss.fit_resample(X,y)

In [195]:
X_train, X_test, y_train, y_test = train_test_split(X_oss, y_oss, test_size=0.2, random_state=42)

In [196]:
hist11,model11=build(X_train,y_train)
score = model11.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 32)                1280      
                                                                 
 dense_105 (Dense)           (None, 64)                2112      
                                                                 
 dense_106 (Dense)           (None, 32)                2080      
                                                                 
 dense_107 (Dense)           (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7/7 - 2s - loss: 0.7061 - accuracy: 0.4760 - 2s/epoch - 283ms/step
Epoch 2/10
7/7 - 0s - loss: 0.6503 - accuracy: 0.6731 - 40ms/epoch - 6ms/step
Epoch 3/10
7/7 - 0s - loss:

###RandomUnderSampler

In [197]:
rus = RandomUnderSampler(random_state=17)
X_rus,y_rus=rus.fit_resample(X,y)

In [198]:
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.2, random_state=42)

In [199]:
hist12,model12=build(X_train,y_train)
score = model12.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_108 (Dense)           (None, 32)                1280      
                                                                 
 dense_109 (Dense)           (None, 64)                2112      
                                                                 
 dense_110 (Dense)           (None, 32)                2080      
                                                                 
 dense_111 (Dense)           (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
5/5 - 2s - loss: 0.6893 - accuracy: 0.5147 - 2s/epoch - 357ms/step
Epoch 2/10
5/5 - 0s - loss: 0.6722 - accuracy: 0.6691 - 20ms/epoch - 4ms/step
Epoch 3/10
5/5 - 0s - loss:

###TomekLinks

In [201]:
tml = TomekLinks()
X_tml,y_tml=tml.fit_resample(X,y)

In [202]:
X_train, X_test, y_train, y_test = train_test_split(X_tml, y_tml, test_size=0.2, random_state=42)

In [203]:
hist13,model13=build(X_train,y_train)
score = model13.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_112 (Dense)           (None, 32)                1280      
                                                                 
 dense_113 (Dense)           (None, 64)                2112      
                                                                 
 dense_114 (Dense)           (None, 32)                2080      
                                                                 
 dense_115 (Dense)           (None, 1)                 33        
                                                                 
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7/7 - 3s - loss: 0.7125 - accuracy: 0.4636 - 3s/epoch - 476ms/step
Epoch 2/10
7/7 - 0s - loss: 0.6275 - accuracy: 0.7182 - 27ms/epoch - 4ms/step
Epoch 3/10
7/7 - 0s - loss: